<a href="https://colab.research.google.com/github/mwolff615/COVID-19/blob/master/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install plotly==4.6.0
!pip install plotly_express==0.4.1

     |████████████████████████████████| 7.2MB 2.6MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import json
from urllib.request import urlopen
import numpy as no
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import sqlite3
import pandas as pd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

#connect to census database
conn = sqlite3.connect('/content/drive/My Drive/VIZ/census.db')
curs = conn.cursor()

print(pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn))

    type  ...                                                sql
0  table  ...             CREATE TABLE sqlite_sequence(name,seq)
1  table  ...  CREATE TABLE tCounty(\n            county_id I...
2  table  ...  CREATE TABLE tHousing(\n             county_id...
3  table  ...  CREATE TABLE tEmployment(\n            county_...
4  table  ...  CREATE TABLE tIncome(\n            county_id I...
5  table  ...  CREATE TABLE "tCOVID" (\n"state" TEXT,\n  "las...

[6 rows x 5 columns]


In [0]:
#create a dataframe from the data found here: https://nssac.bii.virginia.edu/covid-19/dashboard/
ncov_final = pd.DataFrame()
#read each file into the dataframe 
'''for filename in os.listdir('./assignment_6_data'):
    #print(filename)
    #print(type(filename))
    
    name = './assignment_6_data/' + str(filename)
    #ignore files that are not csvs
    if(name == './assignment_6_data/.ipynb_checkpoints'): 
        continue
    else:
        dataframe = pd.read_csv(name, sep = ",", engine='python')
    #clean up the data
    dataframe = dataframe[dataframe['Region'] == 'USA']
    dataframe = dataframe.drop(columns = 'Region')
    dataframe = dataframe[dataframe['name'] != 'Diamond Princess']
    dataframe = dataframe[dataframe['name'] != 'Grand Princess']
    dataframe = dataframe[dataframe['name'] != 'Wuhan Evacuee']
    dataframe = dataframe[dataframe['name'] != 'Guam']
    dataframe = dataframe[dataframe['name'] != 'Puerto Rico']
    dataframe = dataframe[dataframe['name'] != 'United States Virgin Islands']
    dataframe = dataframe[dataframe['name'] != 'USA (Aggregate Recovered)']
    dataframe = dataframe[dataframe['name'] != 'District of Columbia']
    #dataframe["Last Update"] 
    #print(dataframe)
    #rename the columns of the dataframe for a later step
    dataframe.rename(columns = {'name':'state', 'Last Update':'last_date', 'Confirmed':'confirmed', 'Deaths':'deaths',
                              'Recovered':'recovered'}, inplace = True) 
    #add to the dataframe
    ncov_final = ncov_final.append(dataframe)'''
dataframe =  pd.read_csv("/content/drive/My Drive/VIZ/nssac-ncov-sd-04-06-2020.csv", sep = ",", engine='python')
dataframe = dataframe[dataframe['Region'] == 'USA']
dataframe = dataframe.drop(columns = 'Region')
dataframe = dataframe[dataframe['name'] != 'Diamond Princess']
dataframe = dataframe[dataframe['name'] != 'Grand Princess']
dataframe = dataframe[dataframe['name'] != 'Wuhan Evacuee']
dataframe = dataframe[dataframe['name'] != 'Guam']
dataframe = dataframe[dataframe['name'] != 'Puerto Rico']
dataframe = dataframe[dataframe['name'] != 'United States Virgin Islands']
dataframe = dataframe[dataframe['name'] != 'USA (Aggregate Recovered)']
dataframe = dataframe[dataframe['name'] != 'District of Columbia']
dataframe.rename(columns = {'name':'state', 'Last Update':'last_date', 'Confirmed':'confirmed', 'Deaths':'deaths',
                              'Recovered':'recovered'}, inplace = True) 
ncov_final = dataframe

In [0]:
curs2 = conn.cursor()
curs2.execute("PRAGMA foreign_keys = ON")

In [0]:
#create a table of the data read from UVA's database
curs2.execute("DROP TABLE IF EXISTS tCOVID")
sql = """CREATE TABLE tCOVID(
        state TEXT NOT NULL,
        last_date TEXT NOT NULL, 
        confirmed INTEGER CHECK ((typeof(confirmed) == 'integer') AND confirmed >= 0),
        deaths INTEGER CHECK ((typeof(deaths) == 'integer') AND deaths >= 0),
        recovered INTEGER CHECK ((typeof(recovered) == 'integer') AND recovered >= 0), 
        PRIMARY KEY (state, last_date)
        );"""
curs2.execute(sql)
conn.commit()

In [0]:
ncov_final.to_sql('tCOVID', conn, if_exists='replace', index = False)
print(pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn))
sql0 = "SELECT * FROM tCounty"
sql1 = "SELECT * FROM tHousing"
sql2 = "SELECT * FROM tEmployment"
sql3 = "SELECT * FROM tIncome"
df = pd.read_sql_query(sql0, conn)
df.to_csv('tCounty.csv', sep=',', encoding='utf-8')
df = pd.read_sql_query(sql1, conn)
df.to_csv('tHousing.csv', sep=',', encoding='utf-8')
df = pd.read_sql_query(sql2, conn)
df.to_csv('tEmployment.csv', sep=',', encoding='utf-8')
df = pd.read_sql_query(sql3, conn)
df.to_csv('tIncome.csv', sep=',', encoding='utf-8')


    type  ...                                                sql
0  table  ...             CREATE TABLE sqlite_sequence(name,seq)
1  table  ...  CREATE TABLE tCounty(\n            county_id I...
2  table  ...  CREATE TABLE tHousing(\n             county_id...
3  table  ...  CREATE TABLE tEmployment(\n            county_...
4  table  ...  CREATE TABLE tIncome(\n            county_id I...
5  table  ...  CREATE TABLE "tCOVID" (\n"state" TEXT,\n  "las...

[6 rows x 5 columns]


In [0]:
#function to get the percentage of state population w/covid 
def getPercentCovid():
    curs.execute("DROP VIEW IF EXISTS vAllData")
    curs.execute("""CREATE VIEW vAllData AS SELECT DISTINCT B.state, B.county, A.pop, A.year FROM tEmployment AS A
    JOIN tCounty AS B
        ON A.county_id = B.county_id""")
    curs.execute("DROP VIEW IF EXISTS vPopulation")
    curs.execute("""CREATE VIEW vPopulation AS SELECT state, year, sum(pop) AS totalPop
            FROM vAllData
            WHERE year = 2017
            GROUP BY state 
            ORDER BY state""")
    
    curs.execute("DROP VIEW IF EXISTS vCOVID")
    curs.execute("CREATE VIEW vCOVID AS SELECT * FROM vPopulation AS A JOIN tCOVID AS B ON A.state LIKE B.state ")

    sql2 = "SELECT state, confirmed, totalPop, ((100.0*confirmed)/totalPop) as prop FROM vCOVID"
    df = pd.read_sql_query(sql2, conn)
    df.to_csv('percentcovid.csv', sep=',', encoding='utf-8')
    return pd.read_sql(sql2, conn)

In [0]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
df = getPercentCovid()
state = pd.read_csv("/content/drive/My Drive/VIZ/stateabb.csv")
locList = state['abb'].tolist()
percList = df['prop'].tolist()
y = []
for x in percList:
  y.append(np.log(x))
print(percList)
print(locList)
print(y)
#fig = go.Figure(data = go.Choropleth())
print(len(percList))
print(len(locList))

#fig = px.choropleth(locations = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
#, locationmode = 'USA-states', color = [0.06460020857340562, 0.04776609588333712, 0.04171113832619397, 0.06431478396558632, 0.04889034421008677, 0.1288725887483035, 0.19316577776370872, 0.08606545322476895, 0.07424979301742922, 0.1109451085005971, 0.03216613330998182, 0.1276617948019271, 0.12815510819887668, 0.1327524518523988, 0.06709751144672736, 0.0556935199875475, 0.05475333479342226, 0.4844672573120669, 0.06328279164616422, 0.0874729823096976, 0.22174935404413168, 0.2294810600746025, 0.029869780535381474, 0.16704888630871942, 0.07408367305888901, 0.05833577211979319, 0.044723700148916516, 0.08671580877369393, 0.06942044821327954, 0.5162173880129657, 0.04752576597216087, 0.851418494381756, 0.04137147673000216, 0.06288391083243716, 0.051284093838094985, 0.06591524686942, 0.03576915232132756, 0.12045305992982003, 0.11008955223880597, 0.05861168400607571, 0.10284979644311121, 0.10066890929545619, 0.0386221031304922, 0.08106224368169054, 0.37551798745828596, 0.05967844144296582, 0.14677822350918715, 0.05807562624883638, 0.06544993728654716, 0.14277351836781313], scope = 'usa')
#fig.show()
fig = px.choropleth(locations = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
, locationmode = 'USA-states', color = y, scope = 'usa') 
fig.show()

[0.06460020857340562, 0.04776609588333712, 0.04171113832619397, 0.06431478396558632, 0.04889034421008677, 0.1288725887483035, 0.19316577776370872, 0.08606545322476895, 0.07424979301742922, 0.1109451085005971, 0.03216613330998182, 0.1276617948019271, 0.12815510819887668, 0.1327524518523988, 0.06709751144672736, 0.0556935199875475, 0.05475333479342226, 0.4844672573120669, 0.06328279164616422, 0.0874729823096976, 0.22174935404413168, 0.2294810600746025, 0.029869780535381474, 0.16704888630871942, 0.07408367305888901, 0.05833577211979319, 0.044723700148916516, 0.08671580877369393, 0.06942044821327954, 0.5162173880129657, 0.04752576597216087, 0.851418494381756, 0.04137147673000216, 0.06288391083243716, 0.051284093838094985, 0.06591524686942, 0.03576915232132756, 0.12045305992982003, 0.11008955223880597, 0.05861168400607571, 0.10284979644311121, 0.10066890929545619, 0.0386221031304922, 0.08106224368169054, 0.37551798745828596, 0.05967844144296582, 0.14677822350918715, 0.05807562624883638, 0.0

In [0]:

#get the proportion of state residents infected with the novel coronavirus 
getPercentCovid()

,state,confirmed,totalPop,prop
0,Alabama,1841,2849836.0,0.064600
1,Alaska,185,387304.0,0.047766
2,Arizona,2269,5439794.0,0.041711
3,Arkansas,854,1327844.0,0.064315
4,California,15201,31092029.0,0.048890
5,Colorado,4950,3841003.0,0.128873
6,Connecticut,5675,2937891.0,0.193166
7,Delaware,673,781963.0,0.086065
8,Florida,12350,16633043.0,0.074250
9,Georgia,6742,6076879.0,0.110945


In [0]:
#resources
#https://stackoverflow.com/questions/44549110/python-loop-through-excel-sheets-place-into-one-df
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
#https://stackoverflow.com/questions/40835287/python-error-tokenizing-data-c-error-calling-readnbytes-on-source-failed-wi
#https://www.google.com/search?client=firefox-b-1-d&q=pass+vs+continue+python
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
#https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
#https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
#https://www.geeksforgeeks.org/python-pandas-dataframe-append/
#https://datatofish.com/pandas-dataframe-to-sql/
#https://www.google.com/search?client=firefox-b-1-d&q=date+field+sqlite+3
#https://datatofish.com/pandas-dataframe-to-sql/
#https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
#https://matplotlib.org/3.2.1/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
#https://nssac.bii.virginia.edu/covid-19/dashboard/
